<a href="https://colab.research.google.com/github/bochka2005/Python-programming/blob/main/%D0%94%D0%BE%D0%BF.%20%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F/%D0%94%D0%BE%D0%BF%D0%BE%D0%BB%D0%BD%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5%20%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%20%E2%84%962.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [25]:
import time

def timer_decorator(func):
    def wrapper(n):
        start_time = time.perf_counter()
        result = func(n)
        end_time = time.perf_counter()
        print(f"Функция {func.__name__} выполнилась за {end_time - start_time:.6f} секунд для N = {n}")
        return result
    return wrapper

@timer_decorator
def even_numbers_append(n):
    result = []
    for i in range(n + 1):
        if i % 2 == 0:
            result.append(i)
    return result

@timer_decorator
def even_numbers_comprehension(n):
    return [i for i in range(n + 1) if i % 2 == 0]

if __name__ == "__main__":
    N = 1000000

    list1 = even_numbers_append(N)
    print(f"Длина списка: {len(list1)}")

    list2 = even_numbers_comprehension(N)
    print(f"Длина списка: {len(list2)}")

    print(f"Списки одинаковы: {list1 == list2}")

Функция even_numbers_append выполнилась за 0.040418 секунд для N = 1000000
Длина списка: 500001
Функция even_numbers_comprehension выполнилась за 0.032128 секунд для N = 1000000
Длина списка: 500001
Списки одинаковы: True


---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [6]:
def fibonacci_cache(func):
    cache = [0, 1]

    def wrapper(n):
        if n < len(cache):
            return cache[:n + 1]

        for i in range(len(cache), n + 1):
            next_val = cache[i - 1] + cache[i - 2]
            cache.append(next_val)
        return cache[:n + 1]
    return wrapper

@fibonacci_cache
def fibonacci_sequence(n):
    if n == 0:
        return [0]
    elif n == 1:
        return [0, 1]
    return None

print(fibonacci_sequence(5))
print(fibonacci_sequence(3))
print(fibonacci_sequence(7))

[0, 1, 1, 2, 3, 5]
[0, 1, 1, 2]
[0, 1, 1, 2, 3, 5, 8, 13]


---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [8]:
import time

def fibonacci_cache(func):
    cache = [0, 1]

    def wrapper(n):
        if n < len(cache):
            return cache[:n + 1]

        for i in range(len(cache), n + 1):
            next_val = cache[i - 1] + cache[i - 2]
            cache.append(next_val)
        return cache[:n + 1]
    return wrapper

def timer(func):
    def wrapper(n):
        start = time.perf_counter()
        result = func(n)
        end = time.perf_counter()
        print(f"{func.__name__}({n}): {end-start:.6f} сек.")
        return result
    return wrapper

@timer
@fibonacci_cache
def fibonacci_cached(n):
    if n == 0:
        return [0]
    elif n == 1:
        return [0, 1]
    return None

@timer
def fibonacci_uncached(n):
    if n == 0:
        return [0]
    elif n == 1:
        return [0, 1]

    sequence = [0, 1]
    for i in range(2, n + 1):
        sequence.append(sequence[i - 1] + sequence[i - 2])
    return sequence

print("Без кэширования:")
fibonacci_uncached(30)
fibonacci_uncached(30)

print("С кэшированием:")
fibonacci_cached(30)
fibonacci_cached(30)
fibonacci_cached(25)
fibonacci_cached(35)

Без кэширования:
fibonacci_uncached(30): 0.000009 сек.
fibonacci_uncached(30): 0.000004 сек.
С кэшированием:
wrapper(30): 0.000007 сек.
wrapper(30): 0.000001 сек.
wrapper(25): 0.000001 сек.
wrapper(35): 0.000002 сек.


[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229,
 832040,
 1346269,
 2178309,
 3524578,
 5702887,
 9227465]

---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [9]:
def make_multiplier(n):
    def multiplier(x):
        return x * n
    return multiplier

times3 = make_multiplier(3)
print(times3(5))

times5 = make_multiplier(5)
print(times5(4))

15
20


---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [11]:
def rounder(n):
    def round_number(x):
        return round(x, n)
    return round_number

round_to_2 = rounder(2)
print(round_to_2(3.14159))

3.14


---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [17]:
import time

def time_threshold(threshold=0.5):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            result = func(*args, **kwargs)
            end = time.perf_counter()
            elapsed = end - start

            if elapsed > threshold:
                print(f"{func.__name__} выполнилась за {elapsed:.3f} сек., порог: {threshold} сек.")

            return result
        return wrapper
    return decorator

@time_threshold(0.01)
def test():
    time.sleep(0.1)

test()

test выполнилась за 0.100 сек., порог: 0.01 сек.


---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [19]:
def string_generator(char, times):
    for i in range(1, times + 1):
        yield char * i

for s in string_generator('*', 5):
    print(s)

*
**
***
****
*****


---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [20]:
def infinite_sequence():
    n = 1
    while True:
        yield n
        n += 1

gen = infinite_sequence()
for _ in range(5):
    print(next(gen))

1
2
3
4
5


---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [21]:
def combined_lists(lst1, lst2):
    for i in range(min(len(lst1), len(lst2))):
        yield lst1[i]
        yield lst2[i]

for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)

1
a
2
b
3
c


---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [22]:
def reverse_string(s):
    for char in reversed(s):
        yield char

for char in reverse_string('hello'):
    print(char)

o
l
l
e
h


---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [23]:
def powers_of_two(n):
    for i in range(n + 1):
        yield 2 ** i

for num in powers_of_two(5):
    print(num)

1
2
4
8
16
32


---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [24]:
def number_extractor(s):
    current_number = ''
    for char in s:
        if char.isdigit():
            current_number += char
        elif current_number:
            yield int(current_number)
            current_number = ''

    if current_number:
        yield int(current_number)

for num in number_extractor('abc123def45gh6'):
    print(num)

123
45
6


---